Imports

In [146]:
import sys
import logging
import json

Configurando LOGGER

In [147]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Carregando .jsons

In [148]:
with open('map_consultas_relevancias.json', 'r', encoding='utf-8') as f:
    map_consultas_relevancias = json.load(f)

with open('resultados_bm25.json', 'r', encoding='utf-8') as f:
    resultados_bm25 = json.load(f)

with open('resultados_vetorial.json', 'r', encoding='utf-8') as f:
    resultados_tfidf = json.load(f)

In [149]:
def obter_documentos_relevantes_verdadeiros(id_consulta, mapa_relevancias_completo):
    # 1. Acessa o dicionário de relevâncias para a consulta específica
    # O uso de .get() evita erros se o ID não existir
    relevancias_da_consulta = mapa_relevancias_completo.get(str(id_consulta))

    # 2. Se a consulta não existir no mapa, retorna uma lista vazia
    if not relevancias_da_consulta:
        return []

    # 3. Cria uma lista apenas com os IDs dos documentos cujo score é 1 ou 2
    documentos_relevantes = [
        doc_id
        for doc_id, score in relevancias_da_consulta.items()
        if score in [1, 2]
    ]
    
    # 4. Retorna a lista de documentos relevantes encontrados
    return documentos_relevantes

Verificar e retornar correspondência entre consultas relevância e resultados

In [ ]:
def retornar_correspondencia(resultados_modelo, map_consultas_relevancias):
    # 1. Obtém os documentos relevantes verdadeiros
    documentos_relevantes_verdadeiros_raw = obter_documentos_relevantes_verdadeiros(1, map_consultas_relevancias)
    
    # 2. Padroniza os dois conjuntos de dados para string e remove espaços
    #    A função map() aplica uma função (neste caso, str e strip) a cada item da lista.
    
    # Padroniza os resultados do modelo
    set_resultados_modelo = {str(item).strip() for item in resultados_modelo}
    
    # Padroniza os documentos relevantes
    set_documentos_relevantes = {str(item).strip() for item in documentos_relevantes_verdadeiros_raw}

    # 3. Encontra a intersecção entre os dois conjuntos agora padronizados
    correspondencias = set_resultados_modelo.intersection(set_documentos_relevantes)

    # 4. Retorna a lista de correspondências (acertos)
    return list(correspondencias)

BM25 - Precision, Recall e F1

In [151]:
# Calculando acertos
bm25_acertos = retornar_correspondencia(resultados_bm25, map_consultas_relevancias)

# Calcular Precision
precision_bm25 = len(bm25_acertos) / len(resultados_bm25)

LOGGER.info(f"Acertos BM25: {precision_bm25:.4f}")

[2025-07-22 07:43:30,342]INFO(__main__): Acertos BM25: 0.2600
